In [12]:

#LOading all the libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [18]:
#Fetching dataset lfw
#Made a function for loading dataset
def load_lfw_dataset(min_faces_per_person=50, resize=0.5):
    lfw_people = fetch_lfw_people(min_faces_per_person=min_faces_per_person, resize=resize)
    X = lfw_people.data
    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]
    
    print(f"Dataset dimensions: {X.shape}")
    print(f"Number of classes: {n_classes}")
    
    return X, y, target_names, n_classes

#X, y, target_names, n_classes = load_lfw_dataset()#variables
data=fetch_lfw_people(resize=0.4,min_faces_per_person=40,funneled=True)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29440/4061228930.py", line 16, in <module>
    data=fetch_lfw_people(resize=0.4,min_faces_per_person=40,funneled=True)
  File "/home/aaliya/.local/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/aaliya/.local/lib/python3.8/site-packages/sklearn/datasets/_lfw.py", line 353, in fetch_lfw_people
    faces, target, target_names = load_func(
  File "/home/aaliya/.local/lib/python3.8/site-packages/joblib/memory.py", line 577, in __call__
    return self._cached_call(args, kwargs, shelving=False)[0]
  File "/home/aaliya/.local/lib/python3.8/site-packages/joblib/memory.py", line 532, in _cached_call
    return self._call(call_id, args, kwargs, shelving)
  File "/home/aaliya/.local/lib/python3.8/si

In [ ]:
#Applying pca and splitting dataset
def preprocess_and_split_data(X, y, n_components=200, test_size=0.3, random_state=42):
    # Standardizing data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply PCA
    pca = PCA(n_components=n_components, whiten=True, random_state=random_state)
    X_pca = pca.fit_transform(X_scaled)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=test_size, random_state=random_state, stratify=y)
    
    return X_train, X_test, y_train, y_test, pca

X_train, X_test, y_train, y_test, pca = preprocess_and_split_data(X, y)

In [3]:
def create_and_train_mlp(X_train, y_train, hidden_layer_sizes=(100,), max_iter=200, alpha=0.0001, learning_rate_init=0.001):
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, alpha=alpha, 
                        learning_rate_init=learning_rate_init, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp

def plot_learning_curve(mlp):
    plt.figure(figsize=(10, 5))
    plt.plot(mlp.loss_curve_)
    plt.title("Learning Curve")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

# Fine-tuning with different hyperparameters
hidden_layer_sizes_list = [(50,), (100,), (50, 50), (100, 50)]
max_iter_list = [100, 200, 300]
alpha_list = [0.0001, 0.001, 0.01]
learning_rate_init_list = [0.001, 0.01, 0.1]

best_score = 0
best_params = {}

for hidden_layer_sizes in hidden_layer_sizes_list:
    for max_iter in max_iter_list:
        for alpha in alpha_list:
            for learning_rate_init in learning_rate_init_list:
                mlp = create_and_train_mlp(X_train, y_train, hidden_layer_sizes, max_iter, alpha, learning_rate_init)
                score = mlp.score(X_test, y_test)
                
                if score > best_score:
                    best_score = score
                    best_params = {
                        'hidden_layer_sizes': hidden_layer_sizes,
                        'max_iter': max_iter,
                        'alpha': alpha,
                        'learning_rate_init': learning_rate_init
                    }
                
                print(f"Params: {hidden_layer_sizes}, {max_iter}, {alpha}, {learning_rate_init} - Score: {score:.4f}")

print(f"\nBest parameters: {best_params}")
print(f"Best score: {best_score:.4f}")

# Train the best model
best_mlp = create_and_train_mlp(X_train, y_train, **best_params)
plot_learning_curve(best_mlp)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29440/383497687.py", line 28, in <module>
    mlp = create_and_train_mlp(X_train, y_train, hidden_layer_sizes, max_iter, alpha, learning_rate_init)
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/usr/local/lib/p

In [4]:
def create_and_train_mlp_with_regularization(X_train, y_train, alpha, l1_ratio=0.5, **kwargs):
    mlp = MLPClassifier(alpha=alpha, **kwargs)
    mlp.fit(X_train, y_train)
    return mlp

# Try different regularization strengths//alpha
alpha_list = [0.0001, 0.001, 0.01, 0.1]
best_reg_score = 0
best_reg_params = {}

for alpha in alpha_list:
    mlp = create_and_train_mlp_with_regularization(X_train, y_train, alpha, **best_params)
    score = mlp.score(X_test, y_test)
    
    if score > best_reg_score:
        best_reg_score = score
        best_reg_params = {'alpha': alpha, **best_params}
    
    print(f"Alpha: {alpha} - Score: {score:.4f}")

print(f"\nBest regularized parameters: {best_reg_params}")
print(f"Best regularized score: {best_reg_score:.4f}")

# Train the best regularized model
best_reg_mlp = create_and_train_mlp_with_regularization(X_train, y_train, **best_reg_params)
plot_learning_curve(best_reg_mlp)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29440/2875545200.py", line 12, in <module>
    mlp = create_and_train_mlp_with_regularization(X_train, y_train, alpha, **best_params)
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-

In [5]:
# 5. Perform inference

In [6]:
def perform_inference(model, X_test, y_test, target_names):
    y_pred = model.predict(X_test)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=target_names))
    
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

perform_inference(best_reg_mlp, X_test, y_test, target_names)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29440/3530001596.py", line 21, in <module>
    perform_inference(best_reg_mlp, X_test, y_test, target_names)
NameError: name 'best_reg_mlp' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/cor

In [7]:
# Visualize some example predictions

In [8]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return f'predicted: {pred_name}\ntrue: {true_name}'

prediction_titles = [title(best_reg_mlp.predict(X_test), y_test, target_names, i)
                     for i in range(y_test.shape[0])]

plot_gallery(X_test, prediction_titles, h=50, w=37, n_row=3, n_col=4)
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29440/152817698.py", line 17, in <module>
    for i in range(y_test.shape[0])]
NameError: name 'y_test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1140, in structu